<a href="https://colab.research.google.com/github/Mo427-Ashraf/1strepo/blob/main/phase3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# 1. Import Libraries
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, f1_score
from torch.utils.data import DataLoader, TensorDataset

# Set random seed
torch.manual_seed(42)


In [10]:
df = pd.read_csv("customer_data.csv")
print(df.columns)
df.head()


Index(['id', 'age', 'gender', 'income', 'education', 'region',
       'purchase_frequency', 'purchase_amount', 'product_category',
       'promotion_usage', 'satisfaction_score', 'Unnamed: 11',
       'loyalty_status'],
      dtype='object')


,id,age,gender,income,education,region,purchase_frequency,purchase_amount,product_category,promotion_usage,satisfaction_score,Unnamed: 11,loyalty_status
0,1,27,Male,40682,Bachelor,East,frequent,18249,Books,0,6,NaN,Gold
1,2,29,Male,15317,Masters,West,rare,4557,Clothing,1,6,NaN,Regular
2,3,37,Male,38849,Bachelor,West,rare,11822,Clothing,0,6,NaN,Silver
3,4,30,Male,11568,HighSchool,South,frequent,4098,Food,0,7,NaN,Regular
4,5,31,Female,46952,College,North,occasional,19685,Clothing,1,5,NaN,Regular


In [7]:
# 2. Load and Preprocess Dataset

# Load your CSV file (you can upload it via Colab sidebar or use local path)
df = pd.read_csv("customer_data.csv")

# Drop unnecessary columns
df = df.drop(columns=["id"])

# Encode categorical features
cat_cols = ["gender", "education", "region", "purchase_product_cat"]
for col in cat_cols:
    df[col] = LabelEncoder().fit_transform(df[col].astype(str))

# Encode target label
label_encoder = LabelEncoder()
df["loyalty_status"] = label_encoder.fit_transform(df["loyalty_status"])  # e.g., Gold=0, Regular=1, Silver=2

# Separate features and target
X = df.drop("loyalty_status", axis=1)
y = df["loyalty_status"]

# Normalize numeric features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Convert to PyTorch tensors
X_tensor = torch.tensor(X_scaled, dtype=torch.float32)
y_tensor = torch.tensor(y.values, dtype=torch.long)

print("✅ Data preprocessing complete")


KeyError: "['purchase_', '_purchase_product_cat'] not found in axis"

In [ ]:
# 3. Define ANN Model

class ANNModel(nn.Module):
    def __init__(self, input_size, hidden1, hidden2, output_size):
        super(ANNModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden1)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden1, hidden2)
        self.relu2 = nn.ReLU()
        self.output = nn.Linear(hidden2, output_size)

    def forward(self, x):
        x = self.relu1(self.fc1(x))
        x = self.relu2(self.fc2(x))
        return self.output(x)


In [ ]:
# 4. Cross Validation and Training

kf = KFold(n_splits=5, shuffle=True, random_state=42)
fold = 1
accuracies = []
f1_scores = []

for train_idx, val_idx in kf.split(X_tensor):
    print(f"\n----- Fold {fold} -----")

    X_train, X_val = X_tensor[train_idx], X_tensor[val_idx]
    y_train, y_val = y_tensor[train_idx], y_tensor[val_idx]

    train_data = TensorDataset(X_train, y_train)
    val_data = TensorDataset(X_val, y_val)
    train_loader = DataLoader(train_data, batch_size=16, shuffle=True)
    val_loader = DataLoader(val_data, batch_size=16)

    model = ANNModel(input_size=X.shape[1], hidden1=32, hidden2=16, output_size=len(label_encoder.classes_))
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.01)

    # Training loop
    for epoch in range(50):
        model.train()
        for batch_x, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_x)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()

    # Evaluation
    model.eval()
    y_pred, y_true = [], []
    with torch.no_grad():
        for batch_x, batch_y in val_loader:
            outputs = model(batch_x)
            _, predicted = torch.max(outputs.data, 1)
            y_pred.extend(predicted.numpy())
            y_true.extend(batch_y.numpy())

    acc = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred, average='weighted')
    accuracies.append(acc)
    f1_scores.append(f1)

    print(f"Accuracy: {acc:.4f}, F1 Score: {f1:.4f}")
    fold += 1


In [ ]:
# 5. Summary Report

print("\n===== Cross-Validation Summary =====")
print(f"Mean Accuracy: {np.mean(accuracies):.4f}")
print(f"Mean F1 Score: {np.mean(f1_scores):.4f}")
